# 0.0 Imports

In [2]:
import math
import pandas as pd
import inflection

## 0.1. Helper Functions

## 0.2. Loading data

In [12]:
df_sales_0 = pd.read_csv('data/train.csv', low_memory=False)
df_store_0 = pd.read_csv('data/store.csv', low_memory=False)

In [4]:
df_merge = pd.merge(df_sales_0,df_store_0, how='left',on='Store')

# 1.0 Descrição dos dados

## 1.1. Dimensão dos Dados

In [5]:
print( 'Número de Linhas {}'.format(df_merge.shape[0]))
print( 'Número de Colunas {}'.format(df_merge.shape[1]))

Número de Linhas 1017209
Número de Colunas 18


## 1.2. Tipos de Dados

In [7]:
df_merge['Date'] = pd.to_datetime(df_merge['Date'])

In [8]:
df_merge.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

## 1.1. Verificando Nan

In [10]:
df_merge.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

## 1.1. Fillout NA

In [9]:
df_merge['CompetitionDistance'].max()

75860.0

In [18]:
# substituição da feature CompetitionDistance com um valor bem alto
df_merge['CompetitionDistance'] = df_merge['CompetitionDistance'].apply( lambda x: 200000.0 if math.isnan(x) else x)

In [24]:
#CompetitionOpenSinceMonth
df_merge['CompetitionOpenSinceMonth'] = df_merge.apply( lambda x: x['Date'].month if math.isnan( x['CompetitionOpenSinceMonth'] ) else x['CompetitionOpenSinceMonth'], axis=1 )

#CompetitionOpenSinceYear
df_merge['CompetitionOpenSinceYear'] = df_merge.apply( lambda x: x['Date'].year if math.isnan( x['CompetitionOpenSinceYear'] ) else x['CompetitionOpenSinceYear'], axis=1 )

#Promo2SinceWeek
df_merge['Promo2SinceWeek'] = df_merge.apply( lambda x: x['Date'].week if math.isnan( x['Promo2SinceWeek'] ) else x['Promo2SinceWeek'], axis=1 )

#Promo2SinceYear
df_merge['Promo2SinceYear'] = df_merge.apply( lambda x: x['Date'].year if math.isnan( x['Promo2SinceYear'] ) else x['Promo2SinceYear'], axis=1 )

#PromoInterval
month_map = {1: 'Jan',  2: 'Fev',  3: 'Mar',  4: 'Apr',  5: 'May',  6: 'Jun',  7: 'Jul',  8: 'Aug',  9: 'Sep',  10: 'Oct', 11: 'Nov', 12: 'Dec'}

df_merge['PromoInterval'].fillna(0, inplace=True )

# Criando uma nova coluna, informando qual é o mês de referencia
df_merge['month_map'] = df_merge['Date'].dt.month.map( month_map )

# Nova coluna contendo informações se a coluna month_map esta contida em promoInterval
df_merge['is_promo'] = df_merge[['PromoInterval', 'month_map']].apply( lambda x: 0 if x['PromoInterval'] == 0 else 1 if x['month_map'] in x['PromoInterval'].split( ',' ) else 0, axis=1 )

In [26]:
df_merge.isna().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
month_map                    0
is_promo                     0
dtype: int64

In [27]:
df_merge.sample(5).T

,418372,526085,1008360,1003437,596012
Store,1033,591,71,723,273
DayOfWeek,7,1,2,7,1
Date,2014-06-22 00:00:00,2014-03-17 00:00:00,2013-01-08 00:00:00,2013-01-13 00:00:00,2014-01-13 00:00:00
Sales,0,7181,10289,0,4925
Customers,0,623,805,0,581
Open,0,1,1,0,1
Promo,0,1,1,0,0
StateHoliday,0,0,0,0,0
SchoolHoliday,0,0,0,0,0
StoreType,a,a,a,d,a


In [ ]:
#CompetitionDistance
#CompetitionOpenSinceMonth
#CompetitionOpenSinceYear
#Promo2SinceWeek
#Promo2SinceYear
#PromoInterval

In [22]:
df_merge.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                int64
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                        object
dtype: object

In [21]:
df_merge['CompetitionOpenSinceMonth'] = df_merge['CompetitionOpenSinceMonth'].astype(int)
df_merge['CompetitionOpenSinceYear']  = df_merge['CompetitionOpenSinceYear'].astype(int)

df_merge['Promo2SinceWeek']  = df_merge['Promo2SinceWeek'].astype(int)
df_merge['Promo2SinceYear']  = df_merge['Promo2SinceYear'].astype(int)